In [1]:
from sage.crypto.sboxes import PRESENT
from sage.crypto.sbox import SBox
LowMC = SBox(0, 1, 3, 6, 7, 4, 5, 2)

In [13]:
def sbox2transmat(s):
    M = Matrix(GF(2), 2**s.output_size(), 2**s.input_size())
    for x in range(2**s.input_size()):
        M[s(x), x] = 1
    return M

R = PolynomialRing(GF(2), "x")

def matrix_elementary_divisors(M):
    res = []
    for p in (R.gen() * M.matrix_space().identity_matrix() - M).elementary_divisors():
        if p != 1:
            for c in p.factor():
                res.append(c)
    return res

def hypercompanion_matrix(p, m):
    cs = p.coefficients(sparse=False)
    l = len(cs) - 1
    cm = Matrix(GF(2), l, l)
    for i in range(1, l):
        cm[i-1, i] = 1
    for i in range(l):
        cm[l-1, i] = -cs[i]
    hcm = Matrix(GF(2), m*l, m*l)
    for i in range(m):
        hcm[i*l:(i+1)*l,i*l:(i+1)*l] = cm
    for i in range(1,m):
        hcm[i*l-1, i*l] = 1
    return hcm

def decomposed_form(M):
    D = copy(M.matrix_space().zero_matrix())
    i = 0
    for p in matrix_elementary_divisors(M):
        hcm = hypercompanion_matrix(*p)
        s = hcm.nrows()
        D[i:i+s, i:i+s] = hcm
        i += s
    return D, D.is_similar(M, transformation = True)[1]

In [14]:
MLowMC = sbox2transmat(LowMC)
for p in matrix_elementary_divisors(MLowMC):
    print(hypercompanion_matrix(*p), p)
D, P = decomposed_form(MLowMC)
MLowMC == P * D * P.inverse()

[1] (x + 1, 1)
[1] (x + 1, 1)
[1 1]
[0 1] (x + 1, 2)
[0 1 0 0]
[1 1 1 0]
[0 0 0 1]
[0 0 1 1] (x^2 + x + 1, 2)


True

In [15]:
MPRESENT = sbox2transmat(PRESENT)
for p in matrix_elementary_divisors(MPRESENT):
    print(hypercompanion_matrix(*p), p)
D, P = decomposed_form(MPRESENT)
MPRESENT == P * D * P.inverse()

[1] (x + 1, 1)
[1] (x + 1, 1)
[1 1]
[0 1] (x + 1, 2)
[1 1 0 0]
[0 1 1 0]
[0 0 1 1]
[0 0 0 1] (x + 1, 4)
[0 1]
[1 1] (x^2 + x + 1, 1)
[0 1 0]
[0 0 1]
[1 1 0] (x^3 + x + 1, 1)
[0 1 0]
[0 0 1]
[1 0 1] (x^3 + x^2 + 1, 1)


True

In [19]:
P

[1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]
[0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0]
[1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0]
[0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0]
[0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]
[0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]

In [17]:
D

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1]